In [1]:
import scipy
import scipy.io as sio
import numpy as np
from PIL import Image
#import matplotlib.pyplot as plt
import math
import os
from keras.models import Sequential
from keras.layers import *
from keras.layers.advanced_activations import PReLU
#utilities help us transform our data
from keras.utils import * 
from sklearn.utils import shuffle
from sklearn.cross_validation import train_test_split
rows, cols = 48,48
import pandas as pd
from keras.callbacks import EarlyStopping, ModelCheckpoint, CSVLogger
import tensorflow as tf

#!/usr/bin/python
import cv2
from keras.models import load_model
from statistics import mode
#from utils import preprocess_input



Using TensorFlow backend.


In [2]:
#faces,gender_labels = load_images()


In [3]:
class Data_Generator:
    
    def __init__(self, metadata_path, batch_size, val_split):

        self.data = scipy.io.loadmat(metadata_path)
        self.img_paths = self.data['wiki']['full_path'][0][0][0]
        self.genders = self.data['wiki']['gender'][0][0][0]
        self.batch_size = batch_size
        self.val_split = val_split  
        self.sorted_gender = []
        #self.faces = []
        self.face_score = self.data['wiki']['face_score'][0][0][0]
        self.second_face_score = self.data['wiki']['second_face_score'][0][0][0]
        self.load_keys()
        
        
    def load_keys(self):
        indices = []

        for i in range(len(self.face_score)):
            
            data = self.face_score[i]

        # second face score
            data_second_face_score = self.second_face_score[i]

            # if face_score is infinity, image is completely black 

            if (math.isinf(data) == False):

                # if face score is greater than 3 and second face score is nan then image is somewhat better 
                # Looks readable
                if (data > 3.30000000000 and (math.isnan(data_second_face_score) == True)):
                    if (math.isnan(self.genders[i]) == False):
                        indices.append(i)

        self.number_of_imgs = len(indices)
        self.val_size = int(self.number_of_imgs * self.val_split)
        self.train_size = self.number_of_imgs - self.val_size
        self.train_keys = indices[:self.train_size]
        self.val_keys = indices[self.train_size:]



    def load_data(self, is_train):

        while 1:
            faces = []
            gender_labels = []   

            shuffle(self.train_keys)
            keys = self.train_keys
            if not is_train:
                keys = self.val_keys 

            for key in keys:
                img_path = self.img_paths[key][0]
                location_of_image = './wiki_crop/' + img_path
                #img = cv2.imread('./wiki_crop/' + img_path, 0)            
                gender_labels.append(self.genders[key])
                #faces.append(cv2.resize(img, (64, 64)))

                image = Image.open(location_of_image)
                image_resized = image.resize((rows,cols))
                image_gray = image_resized.convert('L') 
                face = np.asarray(image_gray)
                faces.append(face)

                if len(faces) == self.batch_size:
                    faces = np.asarray(faces)
                    faces = np.expand_dims(faces,-1)
                    gender_labels = pd.get_dummies(gender_labels).as_matrix()
                    yield (faces, gender_labels)
                    faces = []
                    gender_labels = []
            faces = np.asarray(faces)
            faces = np.expand_dims(faces,-1)
            gender_labels = pd.get_dummies(gender_labels).as_matrix()
            yield (faces, gender_labels)
                

In [5]:

data_path = './wiki_crop/wiki.mat'
model_save_path = './wiki_crop/simpler_Combined_CNN.hdf5'
num_classes = 2
image_size = (48, 48, 1)
batch_size = 150
num_epochs = 12
val_split = 0.1
data = Data_Generator(data_path, batch_size, val_split)
gen_train = data.load_data(True)
gen_val = data.load_data(False)

model = CNN(image_size, num_classes)
model.compile(optimizer='adam', loss='categorical_crossentropy',
                                        metrics=['accuracy'])
csv_logger = CSVLogger('training.log')
early_stop = EarlyStopping('val_acc', patience=200, verbose=1)
model_checkpoint = ModelCheckpoint(model_save_path,
                                    'val_acc', verbose=0,
                                    save_best_only=True)

model_callbacks = [early_stop, model_checkpoint, csv_logger]

#keras bug 
K.get_session().run(tf.global_variables_initializer())
model.fit_generator(gen_train, nb_epoch=num_epochs, verbose=1, 
                                    validation_data=gen_val,
                                    samples_per_epoch=data.train_size,
                                    nb_val_samples=data.val_size,
                                    callbacks=model_callbacks)

Epoch 1/12
17084/17084 [==============================] - 501s - loss: 0.6095 - acc: 0.7117 - val_loss: 0.5470 - val_acc: 0.7792
Epoch 2/12
17084/17084 [==============================] - 411s - loss: 0.4785 - acc: 0.7776 - val_loss: 0.5736 - val_acc: 0.7086
Epoch 3/12
17084/17084 [==============================] - 421s - loss: 0.4293 - acc: 0.8103 - val_loss: 0.5820 - val_acc: 0.7223
Epoch 4/12
17084/17084 [==============================] - 586s - loss: 0.3973 - acc: 0.8275 - val_loss: 0.3704 - val_acc: 0.8356
Epoch 5/12
17084/17084 [==============================] - 604s - loss: 0.3779 - acc: 0.8388 - val_loss: 0.4674 - val_acc: 0.7882
Epoch 6/12
17084/17084 [==============================] - 453s - loss: 0.3573 - acc: 0.8475 - val_loss: 0.4105 - val_acc: 0.8161
Epoch 7/12
17084/17084 [==============================] - 403s - loss: 0.3395 - acc: 0.8573 - val_loss: 0.3389 - val_acc: 0.8546
Epoch 8/12
17084/17084 [==============================] - 411s - loss: 0.3292 - acc: 0.8653 - val

In [4]:
def CNN(input_shape,num_classes):
    model = Sequential()

    model.add(Convolution2D(16, 7, 7, border_mode='same',
                            input_shape=input_shape))
    model.add(PReLU())
    model.add(BatchNormalization())
    model.add(AveragePooling2D(pool_size=(5, 5),strides=(2, 2), border_mode='same'))
    model.add(Dropout(.5))

    model.add(Convolution2D(32, 5, 5, border_mode='same'))
    model.add(PReLU())
    model.add(BatchNormalization())
    model.add(AveragePooling2D(pool_size=(3, 3),strides=(2, 2), border_mode='same'))
    model.add(Dropout(.5))

    model.add(Convolution2D(32, 3, 3, border_mode='same'))
    model.add(PReLU())
    model.add(BatchNormalization())
    model.add(AveragePooling2D(pool_size=(3, 3),strides=(2, 2), border_mode='same'))
    model.add(Dropout(.5))

    model.add(Flatten())
    model.add(Dense(1028))
    model.add(PReLU())
    model.add(Dropout(0.5))
    model.add(Dense(1028))
    model.add(PReLU())
    model.add(Dropout(0.5))
    model.add(Dense(num_classes))
    model.add(Activation('softmax'))

    return model

In [6]:
def preprocess_image(images):
    
    images = images/255.0
    
    return images

In [ ]:
classification_model_path = './wiki_crop/simpler_Combined_CNN.hdf5'
data_path = './wiki_crop/haarcascade_frontalface_default.xml'
frame_window = 10
gender_labels = {0:'Female',1:'Male'}
face_detection = cv2.CascadeClassifier(data_path)
gender_classifier = load_model(classification_model_path)
gender_window = []

video_capture = cv2.VideoCapture(0)
font = cv2.FONT_HERSHEY_SIMPLEX
cv2.startWindowThread()
cv2.namedWindow('window_frame')

while True:
    _, frame = video_capture.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_detection.detectMultiScale(gray,1.3,5)
    for (x,y,w,h) in faces:
        cv2.rectangle(gray,(x - (int)(0.2*w), y - (int)(0.2*h)),(x + (int)(1.2*w), y + (int)(1.2*h)),(255,0,0),2) # rectangle for image.
        face = gray[y - (int)(0.2*h) :y + (int)(1.2*h),x - (int)(0.2*w) : x + (int)(1.2*w)]
        try:
            face = cv2.resize(face,(48,48))
        except:
            continue
        face = np.expand_dims(face,0)
        face = np.expand_dims(face,-1)
        face = preprocess_image(face)
        #face = face - 143
        gender_arg = np.argmax(gender_classifier.predict(face))
        gender = gender_labels[gender_arg]
        gender_window.append(gender)

        if len(gender_window) >= frame_window:
            gender_window.pop(0)
        try:
            gender_mode = mode(gender_window)
        except:
            continue
        cv2.putText(gray,gender_mode,(x,y-30), font, .7,(255,0,0),1,cv2.LINE_AA)
    try:
        cv2.imshow('window_frame', gray)
    except:
        continue

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video_capture.release()
cv2.destroyAllWindows()